In [14]:
from google.cloud import secretmanager

def get_secret(secret_name):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/trading-assistant-443618/secrets/{secret_name}/versions/latest"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")


In [6]:
import openai
from polygon import RESTClient
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import json
import yfinance as yf


# Set up API keys
OPENAI_API_KEY = get_secret("OPENAI_API_KEY")
XAI_API_KEY = get_secret("XAI_API_KEY")
HYPERBOLIC_API_KEY = get_secret("HYPERBOLIC_API_KEY")

openai.api_key = OPENAI_API_KEY

# Hyperbolic client
open_ai_client = openai.OpenAI(
    api_key=HYPERBOLIC_API_KEY,
    base_url="https://api.hyperbolic.xyz/v1",
    )

xai_client = openai.OpenAI(
    api_key=XAI_API_KEY,
    base_url="https://api.x.ai/v1",
)
XAI_MODEL_NAME = "grok-beta"

functions = [
    {
        "name": "get_live_stock_data",
        "description": "Get live stock data",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "Stock symbol",
                    "example_value": "TSLA",
                },
            },
            "required": ["symbol"],
            "optional": [],
        },
    },
    {
        "name": "get_live_stock_options_chains",
        "description": "Get live stock options chains",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "Stock symbol",
                    "example_value": "TSLA",
                },
            },
            "required": ["symbol"],
            "optional": [],
        },
    },

]

def get_live_stock_data(symbol: str):
    # Create a Ticker object
    stock = yf.Ticker(symbol)

    # Fetch the latest market data
    market_data = stock.history(period="1mo", interval="1d")
    #print(market_data)
    if market_data.empty:
        raise ValueError("No market data found for the symbol.")

    # Get the latest price
    latest_price = market_data['Close'].iloc[-1]

    # Get additional details if needed
    info = stock.info
    company_name = info.get('shortName', 'Unknown Company')
    return {
        "symbol": symbol,
        "market_data": [market_data],
        "company_name": company_name,
        "current_stock_price": latest_price
    }

def get_live_stock_option_chains(symbol: str):
    # Create a Ticker object
    stock = yf.Ticker(symbol)
    # Get current stock price
    current_price = stock.history(period="1d")["Close"].iloc[-1]

    # Define the price range
    lower_bound = current_price * 0.85  # 15% below
    upper_bound = current_price * 1.15  # 15% above

    # Fetch the latest market data
    options_chains = []
    options = stock.options
    for option_date in options[:20]:
      option_chain =  stock.option_chain(option_date)
      calls = option_chain.calls
      puts = option_chain.puts

      # Filter calls and puts
      filtered_calls = calls[(calls["strike"] >= lower_bound) & (calls["strike"] <= upper_bound)]
      filtered_puts = puts[(puts["strike"] >= lower_bound) & (puts["strike"] <= upper_bound)]


      options_chains.append( {
          "option_date": option_date,
          "calls": filtered_calls,
          "puts": filtered_puts
      })



    # Get additional details if needed
    info = stock.info
    company_name = info.get('shortName', 'Unknown Company')
    return {
        "symbol": symbol,
        "company_name": company_name,
        "option_chains": options_chains,
        "current_stock_price": current_price
    }

# Define stock evaluation prompts
def evaluate_stocks(symbol):
    tools = [{"type": "function", "function": f} for f in functions]

    system_prompt = (
        "You are a team of experienced financial analysts that focus on finding the best stocks options for your clients to purchase for short or long term income. "
        "The goal is to help your clients grow their portfolio with about $100k. "
        "Also leverage latest knowledges, discussion and news on social channels such as X, Discord, Trading platforms ... to determine the best stocks and options"
        "Focus on stable growth, great company management, incredible future potential , and/or significant news impact."
        "Proceed with your recommendations and do not ask me to verify or pick my own stock."
        "Use the provided tools to assist the users"
    )
    user_prompt = (
        "Recommend about 5 companies for options trades to grow my porfolio. I am also open to stock investments as well. "
        "Focus on companies that are involved with AI with large market cap and siginificant growth rate that have stable growth, company management and potential, or significant news impact. "
        "Provide reasoning for each recommendation."
    )

    #XAI
    chat_completion = xai_client.chat.completions.create(
        model=XAI_MODEL_NAME,
        messages=[
             {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        tools=tools,
        tool_choice= "auto"
        # temperature=0.7,
        # max_tokens=1024,
    )
    print(chat_completion)
    while chat_completion.choices[0].message.tool_calls and chat_completion.choices[0].message.tool_calls[0].function.name:
      tool_call = chat_completion.choices[0].message.tool_calls[0]
      function_name = tool_call.function.name
      arguments = tool_call.function.arguments
      arguments_json = json.loads(arguments)

      print(chat_completion.choices[0].message.content)
      print("tool_call", tool_call)
      print("function_name", function_name)
      print("arguments", arguments)

      json_data = None
      symbol = arguments_json.get('symbol')
      if function_name == "get_live_stock_options_chains":

        # Call the options chains function
        data = get_live_stock_option_chains(symbol)
        df = pd.DataFrame(data)
        json_data = df.to_json(orient="records")

        user_prompt = (
          "With the provided latest stock options chains, recommend options trades to grow my porfolio. I am also open to stock investments as well. "
          "Provide reasoning for each recommendation."
        )

      elif function_name == "get_live_stock_data":

        # Call the options chains function
        data = get_live_stock_data(symbol)
        df = pd.DataFrame(data)
        json_data = df.to_json(orient="records")

        user_prompt = (
          "With the provided latest stock data, recommend a few options trades to grow my porfolio. I am also open to stock investments as well. "
          "Provide reasoning for each recommendation."
        )

      # Create a message containing the result of the function call
      function_call_result_message = {
          "role": "tool",
          "content": json_data,
          "tool_call_id": tool_call.id
      }

      # Prepare the chat completion call payload

      system_prompt = (
          "You are a team of experienced financial analysts that focus on best way to growth my portfolio based on the provided stocked data for: " + symbol +
          " The goal is to help your clients grow their portfolio with about $100k. "
          "Also leverage latest knowledges, discussion and news on social channels such as X, Discord, Trading platforms ... to determine the best trading strategy"
          "Use the provided tools to assist with your recommendations."
      )

      messages = [
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt},
          chat_completion.choices[0].message,
          function_call_result_message
      ]

      #print("messages", function_call_result_message)

      chat_completion = xai_client.chat.completions.create(
          model=XAI_MODEL_NAME,
          messages=messages,
          tools=tools,
          tool_choice= "auto"
      )

      print(chat_completion)


    return chat_completion.choices[0].message.content


# Main script
if __name__ == "__main__":
    # Example stock symbols (add more symbols based on interest)
    # stock_symbols = [ "PLTR", "TSLA", "NVDA", "SOUN", "SOFI"]
    stock_symbols = [ "TSLA"]

    # Fetch data

    # Evaluate stocks using OpenAI
    for symbol in stock_symbols:
      recommendations = evaluate_stocks(symbol)
      print("\nAI Recommendations :\n", recommendations)


ChatCompletion(id='e1f200a7-7c95-4b86-9901-e85cd8e86856', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='I am calling the `get_live_stock_data` function to gather current market data for AI-related stocks.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='0', function=Function(arguments='{"symbol":"AAPL"}', name='get_live_stock_data'), type='function')]))], created=1733503193, model='grok-beta', object='chat.completion', service_tier=None, system_fingerprint='fp_6ca29cf396', usage=CompletionUsage(completion_tokens=22, prompt_tokens=422, total_tokens=444, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0, text_tokens=422, image_tokens=0)))
I am calling the `get_live_stock_data` function to gather current market data for AI-related stocks.
tool_call ChatCompletionMessageToolCall(id='0', function=Function(argument

In [13]:
# Define stock evaluation prompts
def evaluate_stocks(symbol):
    tools = [{"type": "function", "function": f} for f in functions]

    system_prompt = (
        "You are a team of experienced financial analysts that focus on finding the best trading strategies for this stock: " +symbol +
        "The goal is to help your clients grow their portfolio with about $100k. "
        "Also leverage latest knowledges, discussion and news on social channels such as X, Discord, Trading platforms ... to determine the best stocks and options"
        "Focus on stable growth, great company management, incredible future potential , and/or significant news impact."
        "Proceed with your recommendations and do not ask me to verify or pick my own stock."
        "Use the provided tools to fetch live stock data for recommended stocks to analyze the current market conditions and potential options. Do not ask me if i want to get current stock price or options chains, call the tools provided."
    )
    user_prompt = (
        "Based on the latest stock data and market data, reccommend ways  for options trades to grow my porfolio. I am also open to stock investments as well. "
        "Provide reasoning for each recommendation."
    )

    #XAI
    chat_completion = xai_client.chat.completions.create(
        model=XAI_MODEL_NAME,
        messages=[
             {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        tools=tools,
        tool_choice= "auto"
        # temperature=0.7,
        # max_tokens=1024,
    )
    print(chat_completion)
    while chat_completion.choices[0].finish_reason == "tool_calls":
      tool_call = chat_completion.choices[0].message.tool_calls[0]
      function_name = tool_call.function.name
      arguments = tool_call.function.arguments
      arguments_json = json.loads(arguments)

      print(chat_completion.choices[0].message.content)
      print("tool_call", tool_call)
      print("function_name", function_name)
      print("arguments", arguments)

      json_data = None
      symbol = arguments_json.get('symbol')
      if function_name == "get_live_stock_options_chains":

        # Call the options chains function
        data = get_live_stock_option_chains(symbol)
        df = pd.DataFrame(data)
        json_data = df.to_json(orient="records")

        user_prompt = (
          "With the provided latest stock options chains, recommend  call options trades (2-4 months out) to grow my porfolio. I am also open to stock investments as well. "
          "Provide reasoning for each recommendation."
        )


      elif function_name == "get_live_stock_data":

        # Call the options chains function
        data = get_live_stock_data(symbol)
        df = pd.DataFrame(data)
        json_data = df.to_json(orient="records")

        user_prompt = (
          "With the provided latest stock data, recommend specific long call options trades (2-4 months out) to grow my porfolio. I am also open to stock investments as well. "
          "Provide reasoning for each recommendation. Use the tools to get latest options , proceed without asking me if i want to check specific options."
        )

      # Create a message containing the result of the function call
      function_call_result_message = {
          "role": "tool",
          "content": json_data,
          "tool_call_id": tool_call.id
      }

      # Prepare the chat completion call payload


      messages = [
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt},
          chat_completion.choices[0].message,
          function_call_result_message
      ]

      #print("messages", function_call_result_message)

      chat_completion = xai_client.chat.completions.create(
          model=XAI_MODEL_NAME,
          messages=messages,
          tools=tools,
          tool_choice= "auto"
      )

      print(chat_completion)


    return chat_completion.choices[0].message.content


# Main script
if __name__ == "__main__":
    # Example stock symbols (add more symbols based on interest)
    stock_symbols = [ "TSLA"]
    # stock_symbols = [ "NVDA"]

    # Fetch data

    # Evaluate stocks using OpenAI
    for symbol in stock_symbols:
      recommendations = evaluate_stocks(symbol)
      print("\nAI Recommendations :\n", recommendations)

ChatCompletion(id='33ae6168-3cd4-428c-90a2-a342b0885374', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='I am fetching the latest stock data for Tesla (TSLA) to analyze current market conditions.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='0', function=Function(arguments='{"symbol":"TSLA"}', name='get_live_stock_data'), type='function')]))], created=1733507540, model='grok-beta', object='chat.completion', service_tier=None, system_fingerprint='fp_6ca29cf396', usage=CompletionUsage(completion_tokens=21, prompt_tokens=427, total_tokens=448, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0, text_tokens=427, image_tokens=0)))
I am fetching the latest stock data for Tesla (TSLA) to analyze current market conditions.
tool_call ChatCompletionMessageToolCall(id='0', function=Function(arguments='{"symbol":"TSLA"}